In [ ]:
%matplotlib inline

In [ ]:
import sys

In [ ]:
import uproot
import numpy as np
import awkward

import pandas

import matplotlib.pyplot as plt

In [ ]:
def hits_to_features(hit_data, iev, coll, feats):
    
    if "TrackerHit" in coll:
        new_feats = []
        for feat in feats:
            feat_to_get = feat
            if feat == "energy":
                feat_to_get = "eDep"
            new_feats.append((feat, feat_to_get))
    else:
        new_feats = [(f, f) for f in feats]
        
    feat_arr = {f1: hit_data[coll + "." + f2][iev] for f1, f2 in new_feats}

    sdcoll = "subdetector"
    feat_arr[sdcoll] = np.zeros(len(feat_arr["type"]), dtype=np.int32)
    if coll.startswith("ECAL"):
        feat_arr[sdcoll][:] = 0
    elif coll.startswith("HCAL"):
        feat_arr[sdcoll][:] = 1
    elif coll.startswith("MUON"):
        feat_arr[sdcoll][:] = 2
    else:
        feat_arr[sdcoll][:] = 3
    return awkward.Record(feat_arr)

In [ ]:
fi = uproot.open("reco_p8_ee_tt_ecm380_1302.root")
ev = fi["events"]

iev = 1

In [ ]:
collectionIDs = {k: v for k, v in
    zip(fi.get("metadata").arrays("CollectionIDs")["CollectionIDs"]["m_names"][0],
    fi.get("metadata").arrays("CollectionIDs")["CollectionIDs"]["m_collectionIDs"][0])}
collectionIDs_reverse = {v: k for k, v in collectionIDs.items()}

hit_data = {
    "VXDTrackerHits": ev["VXDTrackerHits"].array(),
    "VXDEndcapTrackerHits": ev["VXDEndcapTrackerHits"].array(),
    "ITrackerHits": ev["ITrackerHits"].array(),
    "OTrackerHits": ev["OTrackerHits"].array(),
    "ECALBarrel": ev["ECALBarrel"].array(),
    "ECALEndcap": ev["ECALEndcap"].array(),
    "ECALOther": ev["ECALOther"].array(),
    "HCALBarrel": ev["HCALBarrel"].array(),
    "HCALEndcap": ev["HCALEndcap"].array(),
    "HCALOther": ev["HCALOther"].array(),
    "MUON": ev["MUON"].array(),
}
    
feats = ["position.x", "position.y", "position.z", "energy", "type"]

hit_feature_matrix = []
for col in sorted(hit_data.keys()):
    icol = collectionIDs[col]
    hit_features = hits_to_features(hit_data[col], iev, col, feats)
    hit_feature_matrix.append(hit_features)
    
hit_feature_matrix = awkward.Record({
    k: awkward.concatenate([hit_feature_matrix[i][k] for i in range(len(hit_feature_matrix))]) for k in hit_feature_matrix[0].fields})

In [ ]:
msk_gen = ev["MCParticles/MCParticles.generatorStatus"].array()==1

In [ ]:
px = ev["MCParticles/MCParticles.momentum.x"].array()[msk_gen][iev]
py = ev["MCParticles/MCParticles.momentum.y"].array()[msk_gen][iev]
pz = ev["MCParticles/MCParticles.momentum.z"].array()[msk_gen][iev]
mass = ev["MCParticles/MCParticles.mass"].array()[msk_gen][iev]
charge = ev["MCParticles/MCParticles.charge"].array()[msk_gen][iev]

In [ ]:
df = pandas.DataFrame()
df["px"] = hit_feature_matrix["position.x"].to_numpy()
df["py"] = hit_feature_matrix["position.y"].to_numpy()
df["pz"] = hit_feature_matrix["position.z"].to_numpy()
df["energy"] = 1000*hit_feature_matrix["energy"].to_numpy()
df["plotsize"] = 0
df["subdetector"] = hit_feature_matrix["subdetector"].to_numpy()

In [ ]:
df.loc[df["subdetector"]==0, "plotsize"] = df.loc[df["subdetector"]==0, "energy"]/5
df.loc[df["subdetector"]==1, "plotsize"] = df.loc[df["subdetector"]==1, "energy"]/10
df.loc[df["subdetector"]==2, "plotsize"] = df.loc[df["subdetector"]==2, "energy"]*100
df.loc[df["subdetector"]==3, "plotsize"] = df.loc[df["subdetector"]==3, "energy"]*100

In [ ]:
import plotly.graph_objects as go
import vector

B = 4.0 # magnetic field in T
c = 3e8 # speed of light in m/s
scale = 2500

mc_x = []
mc_y = []
mc_z = []
for irow in range(len(px)):
    # convert to vector
    v = vector.obj(
        px=px[irow],
        py=py[irow],
        pz=pz[irow],
        mass=mass[irow])
    if charge[irow] == 0:
        # pass
        mc_x += [0, np.clip(scale * v.px/v.mag, -2000, 2000)]
        mc_y += [0, np.clip(scale * v.py/v.mag, -2000, 2000)]
        mc_z += [0, np.clip(scale * v.pz/v.mag, -4000, 4000)]
    else:
        # radius of the helix in m
        R = v.pt / (np.abs(charge[irow]) * 0.3 * B)
        # angular frequency
        omega = np.abs(charge[irow]) * 0.3 * B / (v.gamma * v.mass)
        # time values
        t_values = np.linspace(0, 1/(c * v.beta), 50)
        # trajectory in 3D space
        mc_x += list(scale * R * np.cos(omega * c * t_values + v.phi - np.pi/2) - scale * R * np.cos(v.phi - np.pi/2))
        mc_y += list(scale * R * np.sin(omega * c * t_values + v.phi - np.pi/2) - scale * R * np.sin(v.phi - np.pi/2))
        mc_z += list(scale * v.pz * c * t_values / (v.gamma * v.mass))
        
    mc_x += [None]
    mc_y += [None]
    mc_z += [None]


fig = go.Figure(
    data=[
        go.Scatter3d(
            x=np.clip(df["px"], -2000, 2000),
            y=np.clip(df["py"], -2000, 2000),
            z=np.clip(df["pz"], -4000, 4000),
            mode='markers',
            marker=dict(
                size=np.clip(2+2*np.log(df["plotsize"]), 1, 15),
                color=df["subdetector"],
                colorscale='Viridis',
                opacity=0.8,
            )
        ),
        go.Scatter3d(
            x=mc_x,
            y=mc_y,
            z=mc_z,
            mode='lines',
        )
    ],
)
fig.update_traces(marker_line_width=0, selector=dict(type='scatter3d'))
fig.update_layout(
    width=1000,
    height=1000,
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.update_layout(
    scene=dict(
        xaxis=dict(showgrid=False, showticklabels=False, range=[-2000, 2000]),
        yaxis=dict(showgrid=False, showticklabels=False, range=[-4000, 4000]),
        zaxis=dict(showgrid=False, showticklabels=False, range=[-2000, 2000]),
    ),
    scene_camera=dict(
        up=dict(x=0, y=1, z=0),
        center=dict(x=0, y=0, z=0),
        eye=dict(
            x=1.2,
            y=0.8,
            z=0
        )
    ),
    showlegend=False
)

# uncomment this to create a rotating animation
# idx = 0
# for angle in range(0, 360, 1):
#     camera = dict(
#         up=dict(x=0, y=1, z=1),
#         center=dict(x=0, y=0, z=0),
#         eye=dict(x=1.2 * np.cos(np.radians(angle)),
#                  y=0.8,
#                  z=1.2 * np.sin(np.radians(angle)))
#     )
#     fig.update_layout(scene_camera=camera)
#     frame_filename = f"frame_{idx:03d}.png"
#     idx += 1
#     fig.write_image(frame_filename, scale=2)

Now compile the video with
```
ffmpeg -framerate 10 -pattern_type glob -i '*.png' -c:v libx264 -pix_fmt yuv420p out.mp4
```